# [모듈 3.1] 모델 훈련 스텝 개발 (SageMaker Model Building Pipeline 훈련 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 1. 모델 훈련 스텝 개요
- 2. 기본 라이브러리 로딩
- 3. 훈련에 사용할 전처리된 파일을 확인
- 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
- 6. 세이지 메이커 스튜디오에서 확인하기
- 7. 모델 아티펙트 경로 추출
    
---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---



# 1. 모델 훈련 스텝 개요
- 모델 훈련 및 등록 단계의 개발자 가이드 
    - [모델 훈련 스텝](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training)



# 2. 기본 라이브러리 로딩

세이지 메이커 관련 라이브러리를 로딩 합니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()


%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
%store  

Stored variables and their in-db values:
all_pipeline_endpoint_name             -> 'all-pipeline-endpoint-659531'
bucket                                 -> 'sagemaker-ap-northeast-2-057716757052'
claims_data_uri                        -> 's3://sagemaker-ap-northeast-2-057716757052/sagema
customers_data_uri                     -> 's3://sagemaker-ap-northeast-2-057716757052/sagema
hyperparameters                        -> {'scale_pos_weight': '29', 'max_depth': '3', 'eta'
image_uri                              -> '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/sagema
preprocessing_code                     -> 'src/preprocessing.py'
project_prefix                         -> 'sagemaker-webinar-pipeline-base'
sagemaker_model                        -> 'pipelines-rhjhnjotd9da-fraud-basic-createmo-ctpbz
test_preproc_dir_artifact              -> 's3://sagemaker-ap-northeast-2-057716757052/Fraud-
train_model_

# 3. 훈련에 사용할 전처리된 파일을 확인
이후에 훈련에서 사용할 S3의 저장된 전처리 데이터를 확인 합니다.

## 3.1. 데이터 세트 로딩
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
- 로딩힐 데이터 파일이 S3에 있는지 변수의 경로를 확인 합니다. (train_preproc_dir_artifact)

In [2]:
train_preproc_dir_artifact_file = os.path.join(train_preproc_dir_artifact,'train.csv')
print("train_preproc_dir_artifact_file: \n", train_preproc_dir_artifact_file)
train_prep_df = pd.read_csv(train_preproc_dir_artifact_file)
train_prep_df

train_preproc_dir_artifact_file: 
 s3://sagemaker-ap-northeast-2-057716757052/Fraud-Basic-Process-f65211cab06c0afa0f94d059ed66c9e6/output/train/train.csv


,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


#### [참고] AioClientError 발생시

* pip uninstall boto3
* pip uninstall botocore
* pip uninstall aiobotocore

실행후

* pip install boto3==1.17.106
* pip install s3fs==2021.11.1

# 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 상세 사항은 여기에서 확인 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](img/https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)



## 4.1 모델 빌딩 파이프라인 변수 생성

파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.



### 기본 훈련 변수 및 하이퍼파라미터 설정
- XGBoost에 알고리즘에 입력될 하이퍼 파리미터의 값을 설정 합니다.
- scale_pos_weight 의 경우는 현재의 데이터가 레이블(fraud)간 불균형이 있기에, fraud: 1, non-fraud: 0 의 비율을 계산하여 제공합니다.
    - 하이퍼 파라미터 상세 사항은 여기를 보세요. -->  [XGBoost Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [3]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value= 1
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


## 4.2 하이퍼파라미터 세팅

기본 XGBoost 하이퍼파라미터 외에 `scale_pos_weight` 는 레이블이 뷸균형이 되어 있을 경우에, 레이블 값의 가중치를 부여하는 파라미터 입니다. 레이블 0, 1 의 비율에 따라 지정합니다.

In [4]:
from sagemaker.xgboost.estimator import XGBoost


estimator_output_path = f's3://{bucket}/{project_prefix}/training_jobs'
train_instance_count = 1

hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


## 4.3 Estimator 생성



### 모델 학습을 위한 학습단계 정의 

 XGBoost 사용자 알고리즘을 이용하도록 Estimator를 구성합니다. 
 - 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 
- 사용자 훈련 코드 ""xgboost_starter_script.py"
- 훈련이 끝난 후에 결과인 모델 아티펙트의 경로 "estimator_output_path" 지정 합니다. 지정 안할 시에는 디폴트 경로로 저장 됩니다.


- `training_instance_type`, `training_instance_count` 파라미터가 사용된 것을 확인합니다. 

In [5]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version="1.3-1"    
#    framework_version = "1.0-1"
)

## 4.4 모델 훈련 스탭 생성
- 스텝 생성시에 위에서 생성한 Estimator 입력 및 입력 데이타로서 전처리 데이터가 존재하는 S3 경로를 제공합니다.

In [6]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="Fraud-Basic-Train",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_dir_artifact,
            content_type="text/csv"
        ),
    },
)

이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.
- [알림] `6.1.All-Pipeline.ipynb` 노트북에서는 입력을 전처리 스텝의 결과를 지정합니다. 여기서는 전처리 스텝과 독립적으로 실행하기 위해서 S3의 입력 파일 경로를 직접 기술 하였습니다.
-  `6.1.All-Pipeline.ipynb` 에서의 step_train 코드

```python
step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
    },
)    
 ```


# 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행


이제 지금까지 생성한 단계들을 하나의 파이프라인으로 조합하고 실행하도록 하겠습니다.

파이프라인은 name, parameters, steps 속성이 필수적으로 필요합니다. 
여기서 파이프라인의 이름은 (account, region) 조합에 대하여 유일(unique))해야 합니다.
우리는 또한 여기서 Experiment 설정을 추가 하여, 실험에 등록 합니다.

주의:

- 정의에 사용한 모든 파라미터가 존재해야 합니다.
- 파이프라인으로 전달된 단계(step)들은 실행순서와는 무관합니다. SageMaker Pipeline은 단계가 실행되고 완료될 수 있도록 의존관계를를 해석합니다.

## 5.1 파이프라인 정의
파이프라인 정의시에 아래 3개의 인자를 제공합니다.
- 파이프라인 이름
- 파이프라인 파라미터
- 파이프라인 실험 설정
- 스텝 정의 (여기서는 훈련, 모델 등록 두가지 스텝 임)

In [7]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_instance_type,        
        training_instance_count,         
        input_data,
    ],
    steps=[step_train],
)

## 5.2 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [8]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'TrainingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-webinar-pipeline-base/input'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Fraud-Basic-Train',
   'Type': 'Training',
   'Arguments': {'AlgorithmSpecification': {'TrainingInputMode': 'File',
     'TrainingImage': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.3-1'},
    'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-webinar-pipeline-base/training_jobs'},
    'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
    'ResourceConfig': {'InstanceCount': {'Get': 'Paramete

## 5.3 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [9]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [10]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-webinar-pipeline-base',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-webinar-pipeline-base/execution/51fk1ly0rzau',
 'PipelineExecutionDisplayName': 'execution-1647826431115',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2022, 3, 21, 1, 33, 51, 48000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 21, 1, 33, 51, 48000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '184a034b-d948-4baf-bc30-ea2da41a2f42',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '184a034b-d948-4baf-bc30-ea2da41a2f42',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '437',
   'date': 'Mon, 21 Mar 2022 01:33:50 GMT'},
  'RetryAttempts': 0}}

## 5.4 파이프라인 실행 기다리기

In [11]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

## 5.5 파이프라인 실행 단계 기록 보기

In [12]:
execution.list_steps()

[{'StepName': 'Fraud-Basic-Train',
  'StartTime': datetime.datetime(2022, 3, 21, 1, 33, 52, 385000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 3, 21, 1, 37, 6, 205000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-51fk1ly0rzau-fraud-basic-train-hkegzdmk5b'}}}]

# 6. 세이지 메이커 스튜디오에서 확인하기
- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
    - SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)



![train_step_sm.studio.png](img/train_step_sm.studio.png)

# 7. 모델 아티펙트 경로 추출


## 7.1. 훈련 모델 결과의 경로 지정
- 다음 노트북에서 사용할 훈련 모델 아티펙트의 S3 경로를 저장 합니다.

In [13]:
from src.p_utils import get_train_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: \n", train_model_artifact)


 train_model_artifact: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-webinar-pipeline-base/training_jobs/pipelines-51fk1ly0rzau-Fraud-Basic-Train-HKeGZDmK5B/output/model.tar.gz


## 7.2 훈련시 사용한 도커 이미지를 저장
- 추론시에 사용할 도커 이미지를 추론시에도 동일하게 사용하기 위해서 저장 합니다.
    - 일반적으로 훈련, 추론의 도커 이미지를 분리해서 사용합니다. 하지만 간단한 알고리즘의 경우는 훈련, 추론의 도커 이미지를 같이 사용하기도 합니다.

In [14]:
image_uri = xgb_train.image_uri
print("image_uri: \n", image_uri)

image_uri: 
 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.3-1


훈련 모델 아티펙트와, 훈련시 사용한 다커 이미지의 경로를 저장 합니다.

In [15]:
%store train_model_artifact
%store image_uri

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
